# NYCU Machine Learning 2024 : HW4 SVM

In [1]:
import numpy as np
import cvxpy as cp
import pandas as pd

from pathlib import Path
from rich import print
from dataclasses import dataclass
from typing import Callable

In [2]:
LABEL = ["Setosa" , "Versicolor" , "Virginica" ]

COLOR_1 = dict(zip(LABEL, ["red" , "green" , "blue"]))
COLOR_2 = dict(zip(LABEL, ["pink" , "yellow" , "orange"]))
COLOR_3 = dict(zip(LABEL, ["brown", "lightgreen", "navy", "magenta"]))
COLOR_4 = dict(zip(LABEL, ["teal", "gold", "violet", "coral"]))

COLOR_SELECT = {
    "before": {
        "train":COLOR_1,
        "test":COLOR_2,
    },
    "after": {
        "train":COLOR_3,
        "test":COLOR_4,
    }
}

COLUMN_NAME = ["Sepal length", "Sepal width" , "Petal length" , "Petal width" , "Label"]
TRAIN_DATA_SIZE = 25
ASSETS = "./assets"

In [3]:
assets_folder = Path(ASSETS)
assets_folder.mkdir(parents=True, exist_ok=True) 

In [4]:
def load_iris_file(with_name:bool=False)->pd.DataFrame:
    df = pd.read_fwf("./iris.txt")
    
    df_new = pd.DataFrame({k:[v] for k ,v in zip(COLUMN_NAME , df.columns)},dtype=float)
    df.columns = COLUMN_NAME
    df_new = pd.concat([df_new, df], axis=0).reset_index().drop(columns=["index"])
    
    if not with_name:
        return df_new
    
    df_with_name = df_new.copy()
    
    df_with_name["Label"] = df_with_name["Label"].apply(lambda x : LABEL[int(x)-1])
    
    return df_with_name

In [5]:
df = load_iris_file(with_name=True)
df

,Sepal length,Sepal width,Petal length,Petal width,Label
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Virginica
146,6.3,2.5,5.0,1.9,Virginica
147,6.5,3.0,5.2,2.0,Virginica
148,6.2,3.4,5.4,2.3,Virginica


In [6]:
POSITIVE_CLASS ,NEGATIVE_CLASS= "Versicolor" , "Virginica"

In [7]:
df_need = df[["Petal length","Petal width", "Label"]]
df_need

,Petal length,Petal width,Label
0,1.4,0.2,Setosa
1,1.4,0.2,Setosa
2,1.3,0.2,Setosa
3,1.5,0.2,Setosa
4,1.4,0.2,Setosa
...,...,...,...
145,5.2,2.3,Virginica
146,5.0,1.9,Virginica
147,5.2,2.0,Virginica
148,5.4,2.3,Virginica


In [8]:
df_need_class = df_need[(df_need["Label"] == POSITIVE_CLASS) | (df_need["Label"] == NEGATIVE_CLASS)]
df_need_class = df_need_class.reset_index().drop(columns=["index"])
df_need_class

,Petal length,Petal width,Label
0,4.7,1.4,Versicolor
1,4.5,1.5,Versicolor
2,4.9,1.5,Versicolor
3,4.0,1.3,Versicolor
4,4.6,1.5,Versicolor
...,...,...,...
95,5.2,2.3,Virginica
96,5.0,1.9,Virginica
97,5.2,2.0,Virginica
98,5.4,2.3,Virginica


In [9]:
label_to_index = {POSITIVE_CLASS : 1 , NEGATIVE_CLASS : -1}
index_to_label = {1 : POSITIVE_CLASS , -1 : NEGATIVE_CLASS}


label_to_index_np = np.vectorize(label_to_index.get)
index_to_label_np = np.vectorize(index_to_label.get)

In [10]:
dummy_data_x , dummy_data_y = df_need_class.drop(columns=["Label"]).to_numpy() , df_need_class["Label"].to_numpy() 

In [11]:
dummy_data_x.shape

(100, 2)

In [12]:
dummy_data_y = label_to_index_np(dummy_data_y)
dummy_data_y

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

## Model

In [13]:
def rbf(sigma:float) -> Callable[[np.ndarray, np.ndarray], np.ndarray]:
    def run(x_1:np.ndarray, x_2:np.ndarray)->np.ndarray:
        assert x_1.shape[0] == x_2.shape[0], "输入的两个向量必须有相同的特征数"
        over = 1/ (2* np.square(sigma))
        dis = -np.linalg.norm(x_1 - x_2)**2 
        
        return np.exp(dis* over)
    
    return run

def poly(p:int)-> Callable[[np.ndarray, np.ndarray], np.ndarray]:
    def run(x_1:np.ndarray, x_2:np.ndarray)->np.ndarray:
        assert x_1.shape[0] == x_2.shape[0], "输入的两个向量必须有相同的特征数"
        
        return (x_1.T @ x_2)**p
    
    return run

def linear() -> Callable[[np.ndarray, np.ndarray], np.ndarray]:
    
    def run(x_1:np.ndarray, x_2:np.ndarray)->np.ndarray:
        assert x_1.shape[0] == x_2.shape[0], "输入的两个向量必须有相同的特征数"
        
        return x_1.T @ x_2
    
    return run

class Kernel:
    kernel_dict = {
        "linear":linear,
        "rbf": rbf,
        "poly": poly,
    }
        
    @staticmethod
    def get_kernel(name:str, config:dict) -> Callable[[np.ndarray, np.ndarray], np.ndarray]:
        if name not in Kernel.kernel_dict:
            raise NotImplementedError(f"{name} kernel not implemented. Available kernels: {list(Kernel.kernel_dict.keys())}")
        
        func = Kernel.kernel_dict[name]
        
        return func(**config) 

In [14]:
kernel = Kernel.get_kernel("rbf", {"sigma":1})

In [15]:
class SupportVectorMachine:
    def __init__(self, C:int, kernel_name:str="linear", kernel_arg:dict=dict(),threshold:float=1e-20):
        self._c = C
        self._threshold = threshold
        # like ("ay": ... , "x": ...,)
        self._a_x_y :np.ndarray = None
        self._b :float = None
        self._kernel  = Kernel.get_kernel(kernel_name, kernel_arg)
        return
    
    def _build_k_matrix(self, x:np.ndarray)->np.ndarray:
        size = x.shape[0]
        return np.array([[self._kernel(x[i], x[j])  for j in range(size)] for i in range(size) ])
    
    def _find_alpha(self, K:np.ndarray, x:np.ndarray , y:np.ndarray)-> tuple[np.ndarray,np.ndarray]:
        size = x.shape[0] 
        alpha = cp.Variable(size)
        
        big_k = np.diag(y) @ K @ np.diag(y)
        big_k = cp.psd_wrap(big_k)
        
        objective = cp.Minimize((1/2) * cp.quad_form(alpha, big_k) -cp.sum(alpha))
        constraints = [
            cp.sum(cp.multiply(alpha, y)) == 0,  
            alpha >= 0, 
            alpha <= self._c
        ]
        problem = cp.Problem(objective, constraints) 
        problem.solve()
        
        alpha = alpha.value.reshape(-1,1)
        
        # filter the important one 
        support_vectors = np.where((self._c >= alpha) & (alpha > self._threshold))[0]
        
        return alpha , support_vectors
    
    def _find_bias(self,alpha:np.ndarray,support_vectors:np.ndarray, y: np.ndarray, K: np.ndarray):
        res = [y[i] - np.sum([alpha[j]*y[j]*K[i][j] for j in support_vectors]) for i in support_vectors]
        
        return np.mean(res)
    
    def train(self, x:np.ndarray, y:np.ndarray): # [batch, feature]   [batch, 1]
        # get the a
        K = self._build_k_matrix(x) 
        alpha , support_vector = self._find_alpha(K,x,y)
        # find the best b
        self._b = self._find_bias(alpha,support_vector,y,K)
        
        print(self._b)
        
        table = np.hstack((alpha * y.reshape(-1,1), x ))
        
        self._a_x_y = table[support_vector]
        print(self._a_x_y)
        
        return 
    
    def cal_one_item(self, ay:np.ndarray, x_kernel:np.ndarray, x_item: np.ndarray)->np.ndarray:
        # x [1, feature]
        # x_kernel [a, feature]
        
        res = np.sum(ay * self._kernel(x_kernel.T, x_item.reshape(-1,1))) + self._b
        
        return res
    
    def __call__(self, x : np.ndarray)->np.ndarray:
        # x [batch, feature]
        ay, x_kernel = self._a_x_y[:, 0].reshape(-1,1) ,self._a_x_y[:, 1:]
        
        result = [self.cal_one_item(ay, x_kernel=x_kernel, x_item=x_item) for x_item in x]
        
        res = np.hstack(result) 
        return res
    

In [16]:
model = SupportVectorMachine(C =1 , kernel_name="linear" )

In [17]:
model.train(x=dummy_data_x, y=dummy_data_y)

-16.617235772357972

[[ 0.85365853  4.7         1.4       ]
 [ 1.          4.9         1.5       ]
 [ 1.          4.7         1.6       ]
 [ 0.85365854  4.7         1.4       ]
 [ 1.          4.8         1.8       ]
 [ 1.          4.9         1.5       ]
 [ 1.          5.1         1.6       ]
 [ 1.          4.5         1.6       ]
 [ 1.          4.7         1.5       ]
 [-0.35365854  5.1         1.9       ]
 [-1.          4.5         1.7       ]
 [-1.          5.          1.5       ]
 [-1.          4.8         1.8       ]
 [-1.          5.1         1.5       ]
 [-1.          5.6         1.4       ]
 [-1.          4.8         1.8       ]
 [-0.35365853  5.1         1.9       ]
 [-1.          5.          1.9       ]]

In [18]:
model(dummy_data_x)

array([-0.73211382, -1.32089431, -0.03406504, -3.03869919, -0.99918699,
       -1.4301626 , -0.62284553, -5.45455285, -1.10845528, -3.30577236,
       -4.81113821, -2.28601626, -3.20260163, -0.73211382, -4.32552846,
       -1.69723577, -1.32089431, -2.88089431, -1.32089431, -3.4696748 ,
       -0.19186992, -3.03869919, -0.03406504, -0.84138211, -2.07357724,
       -1.69723577, -0.4104065 ,  0.39691057, -1.32089431, -4.81113821,
       -3.79138211, -4.16772358, -3.41504065,  0.66398374, -1.32089431,
       -1.26626016, -0.67747967, -1.75186992, -2.71699187, -3.03869919,
       -1.80650407, -1.05382114, -3.09333333, -5.45455285, -2.39528455,
       -2.4499187 , -2.39528455, -2.07357724, -6.36504065, -2.71699187,
        4.05105691,  0.82788618,  3.51081301,  2.38178862,  3.24373984,
        5.76276423, -1.21162602,  4.63373984,  3.02520325,  4.37276423,
        0.88252033,  1.47130081,  2.22398374,  0.56081301,  1.10105691,
        1.6898374 ,  2.0600813 ,  6.13910569,  6.83715447,  0.28